In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spotify-million-song-dataset/spotify_millsongdata.csv


*Import Dependencies*

In [2]:
import os
import sys
import warnings
import pandas 
import numpy
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
%matplotlib inline

*Load Dataset*

In [3]:
df = pd.read_csv("/kaggle/input/spotify-million-song-dataset/spotify_millsongdata.csv")

In [4]:
df

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...


*EDA*

In [5]:
df.head(4)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...


In [6]:
df.tail(4)

,artist,song,link,text
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [7]:
df.drop("link", axis=1, inplace = True)


In [8]:
df.head()

,artist,song,text
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...


In [9]:
df = df[0:50000]
df.drop_duplicates(subset="song", inplace=True)
df.reset_index(drop = True, inplace = True)

In [10]:
df.rename(columns = {"artist":'Artist','song':'Song','text':'Lyrics'},inplace = True)

In [11]:
df.head(5)

,Artist,Song,Lyrics
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...


In [12]:
df.isnull().sum()

Artist    0
Song      0
Lyrics    0
dtype: int64

In [13]:
df.duplicated().sum()

0

In [14]:
df.columns

Index(['Artist', 'Song', 'Lyrics'], dtype='object')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39466 entries, 0 to 39465
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Artist  39466 non-null  object
 1   Song    39466 non-null  object
 2   Lyrics  39466 non-null  object
dtypes: object(3)
memory usage: 925.1+ KB


In [16]:
df['Combined_features'] = df['Artist']+" "+df['Song']+" "+df['Lyrics']

In [17]:
df.head()

,Artist,Song,Lyrics,Combined_features
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA...","ABBA Ahe's My Kind Of Girl Look at her face, i..."
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen...","ABBA Andante, Andante Take it easy with me, pl..."
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...,ABBA As Good As New I'll never know why I had ...
3,ABBA,Bang,Making somebody happy is a question of give an...,ABBA Bang Making somebody happy is a question ...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...,ABBA Bang-A-Boomerang Making somebody happy is...


In [18]:
df.tail()

,Artist,Song,Lyrics,Combined_features
39461,Ramones,Don't Bust My Chops,I'm sick and tired of you calling me names \r...,Ramones Don't Bust My Chops I'm sick and tired...
39462,Ramones,Don't Come Close,You know it's generally known \r\nYou got eve...,Ramones Don't Come Close You know it's general...
39463,Ramones,Don't Pour Out All The Oil,Verse: \r\nDon't pour out all the oil \r\nIt...,Ramones Don't Pour Out All The Oil Verse: \r\...
39464,Ramones,Eat That Rat,One-two-three-four \r\n \r\nYou don't want t...,Ramones Eat That Rat One-two-three-four \r\n ...
39465,Ramones,Endless Vacation,I kill myself kill the pain \r\nA suicide dea...,Ramones Endless Vacation I kill myself kill th...


In [19]:
df.Combined_features[0]

"ABBA Ahe's My Kind Of Girl Look at her face, it's a wonderful face  \r\nAnd it means something special to me  \r\nLook at the way that she smiles when she sees me  \r\nHow lucky can one fellow be?  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?  \r\n  \r\nAnd when we go for a walk in the park  \r\nAnd she holds me and squeezes my hand  \r\nWe'll go on walking for hours and talking  \r\nAbout all the things that we plan  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?\r\n\r\n"

In [20]:
df.Combined_features[1]

"ABBA Andante, Andante Take it easy with me, please  \r\nTouch me gently like a summer evening breeze  \r\nTake your time, make it slow  \r\nAndante, Andante  \r\nJust let the feeling grow  \r\n  \r\nMake your fingers soft and light  \r\nLet your body be the velvet of the night  \r\nTouch my soul, you know how  \r\nAndante, Andante  \r\nGo slowly with me now  \r\n  \r\nI'm your music  \r\n(I am your music and I am your song)  \r\nI'm your song  \r\n(I am your music and I am your song)  \r\nPlay me time and time again and make me strong  \r\n(Play me again 'cause you're making me strong)  \r\nMake me sing, make me sound  \r\n(You make me sing and you make me)  \r\nAndante, Andante  \r\nTread lightly on my ground  \r\nAndante, Andante  \r\nOh please don't let me down  \r\n  \r\nThere's a shimmer in your eyes  \r\nLike the feeling of a thousand butterflies  \r\nPlease don't talk, go on, play  \r\nAndante, Andante  \r\nAnd let me float away  \r\n  \r\nI'm your music  \r\n(I am your music a

In [21]:
for i in range(0,6):
    print("****************************************************\n")
    print(df.Combined_features[i])

****************************************************

ABBA Ahe's My Kind Of Girl Look at her face, it's a wonderful face  
And it means something special to me  
Look at the way that she smiles when she sees me  
How lucky can one fellow be?  
  
She's just my kind of girl, she makes me feel fine  
Who could ever believe that she could be mine?  
She's just my kind of girl, without her I'm blue  
And if she ever leaves me what could I do, what could I do?  
  
And when we go for a walk in the park  
And she holds me and squeezes my hand  
We'll go on walking for hours and talking  
About all the things that we plan  
  
She's just my kind of girl, she makes me feel fine  
Who could ever believe that she could be mine?  
She's just my kind of girl, without her I'm blue  
And if she ever leaves me what could I do, what could I do?


****************************************************

ABBA Andante, Andante Take it easy with me, please  
Touch me gently like a summer evening breeze  
Tak

*Function for cleaning data*

In [22]:
import re
import nltk
from nltk.corpus import stopwords
def cleaning(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    text = text.lower()
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

In [23]:
df['Combined_features'] = df['Combined_features'].apply(func = cleaning)

In [24]:
for i in range(0,6):
    print("\n***********************************************\n")
    print(df.Combined_features[i])


***********************************************

abba ahe kind girl look face wonderful face means something special look way smiles sees lucky one fellow kind girl makes feel fine could ever believe could mine kind girl without blue ever leaves could could go walk park holds squeezes hand go walking hours talking things plan kind girl makes feel fine could ever believe could mine kind girl without blue ever leaves could could

***********************************************

abba andante andante take easy please touch gently like summer evening breeze take time make slow andante andante let feeling grow make fingers soft light let body velvet night touch soul know andante andante go slowly music music song song music song play time time make strong play cause making strong make sing make sound make sing make andante andante tread lightly ground andante andante oh please let shimmer eyes like feeling thousand butterflies please talk go play andante andante let float away music music s

*Vectorization*

In [25]:
tfv = TfidfVectorizer(max_features = 10000)

*Data Transformation*

In [26]:
tfv_matrix = tfv.fit_transform(df["Combined_features"])

In [27]:
tfv_matrix.shape

(39466, 10000)

*Cosine Similarities Matrix*

In [28]:
cosine_sim = cosine_similarity(tfv_matrix)

In [29]:
song_user_likes = "Hope"
song_index = df[df.Song == song_user_likes].index[0]

In [30]:
song_index

659

In [31]:
similar_songs = list(enumerate(cosine_sim[song_index]))

In [32]:
similar_songs[0:6]

[(0, 0.04092533924868175),
 (1, 0.01746788081317406),
 (2, 0.04942975440682899),
 (3, 0.0065672233749841415),
 (4, 0.007298608643914333),
 (5, 0.0580729402516177)]

In [33]:
similar_song_sorted = sorted(similar_songs,key = lambda x:x[1],reverse = True)

In [34]:
similar_song_sorted[0:6]

[(659, 1.0000000000000002),
 (8068, 0.5784493048716957),
 (37444, 0.5647907063872308),
 (23056, 0.45309671761554454),
 (20793, 0.4242324884462278),
 (20002, 0.3878758836093712)]

In [35]:
for song in similar_song_sorted[1:11]:
    
    similar_songs = df[df.index == song[0]]["Song"].values[0]
    print(similar_songs)

Who Do You Love, I Hope
Here's Hope
I Hope You Find It
Be Alright
I Want To Be With You Tonight
Alright
Hope It's Too Late
If He's Ever Near
Hope And Pray
Want


*Recommender System Creation*

In [36]:
def recommended_songs(song_user_likes):
    song_index = df[df.Song == song_user_likes].index[0]
    similar_songs = list(enumerate(cosine_sim[song_index]))
    similar_song_sorted = sorted(similar_songs, key=lambda x:x[1], reverse=True)
    
    for song in similar_song_sorted[1:11]:
        similar_songs = df[df.index == song[0]]["Song"].values[0]
        print(similar_songs)

In [37]:
recommended_songs("Hope")

Who Do You Love, I Hope
Here's Hope
I Hope You Find It
Be Alright
I Want To Be With You Tonight
Alright
Hope It's Too Late
If He's Ever Near
Hope And Pray
Want


*Thanks*